#### GAT model using the electron transition density atomic contribution matrix
$$
\huge \tilde{\gamma}^{[l,m]}_{AA^{\prime}}
$$

###### Check out the paper of this work <br>
##### *X-ray absorption spectroscopy reveals charge transfer in π-stacked aromatic amino acids*:<br> https://doi.org/10.1039/D4CP04615C

____________________________

In [1]:
def save_ETDAC_matrix(data_dict, data_set_name="data_etdac_matrix.h5"):
    """
    Get the node/edge features for each molecule and save
    all the results in H5PY format.
    Args:
    data_dict (dict) contains a hash (key) and 
     the ETDAC matrix (value) of each molecule.
    data_set_name (str, optional) is the name of the H5PY file to be
     created. By default that file is called "data_etdac_matrix.h5".
    """
    
    with h5py.File(data_set_name, 'w') as f:

        # Get node/edge features for the list of molecules
        for hash in data_dict.keys():
#df.to_hdf('data.h5', key='df', mode='w', format='table')
            qm_group = f.create_group(f"sample_{hash}")
            qm_group.create_dataset("ETDAC_matrix", data=data_dict[hash], compression="gzip")      
            qm_group.attrs["hash"] = hash

In [2]:
import h5py

In [3]:
import numpy as np
import pandas as pd

In [4]:
from ase.io import read
from rdkit import Chem

In [5]:
## Technical functions

In [6]:
def load_etdac_h5py(h5file, hash_mol):
    """
    Load data from one ETDAC matrix (group) from a h5py
    file (h5file) using the specific hash (hash_mol).
    """
    # Load a specific molecule using the hash
    id_mol = f"sample_{hash_mol}"
    with h5py.File(h5file, 'r') as f:
        # Load global parameters
        global_params = dict(f.attrs)
        #print("Global params:", global_params)

        # Access a specific sample using the hash
        sample = f[id_mol]
        df = pd.DataFrame.from_records(sample["ETDAC_matrix"][:])
        # Restore row name
        df.set_index(sample.attrs["index_name"], inplace=True)
        # Restore column name
        df.columns.name = sample.attrs["column_name"]
        # load arrays
        return [
            sample.attrs["hash"],
            df
        ]

In [7]:
def load_nodesnedges_h5py(h5file, hash_mol):
    """
    Load data from one molecule (group) from a h5py
    file (h5file) using the specific hash (hash_mol).
    """
    # Load a specific molecule using the hash
    id_mol = f"sample_{hash_mol}"
    with h5py.File(h5file, 'r') as f:
        # Load global parameters
        global_params = dict(f.attrs)
        print("Global params:", global_params)

        # Access a specific sample using the hash
        sample = f[id_mol]
        # load arrays
        return [
            sample.attrs["name"],
            sample["Node_features"][:],
            sample["Edge_list"][:],
            sample["Edge_features"][:]
        ]

In [8]:
def loadseveral_nodesnedges_h5py(h5file):
    """
    Load data from several molecules (groups) from a h5py
    file (h5file).
    Do not use this function for hundreds/thousands of 
    molecules.
    """
    data = {}
    with h5py.File(h5file, 'r') as mol:
        for group_name in mol.keys(): # iteration through groups (molecules)
            group = mol[group_name]
            df = pd.DataFrame.from_records(group["ETDAC_matrix"][:])
            # Restore row name
            df.set_index(group.attrs["index_name"], inplace=True)
            # Restore column name
            df.columns.name = group.attrs["column_name"]
            # load arrays
            data[group_name] = df
    return data

In [9]:
def save_ETDAC_matrix(data_dict, data_set_name="data_etdac_matrix.h5"):
    """
    Get the node/edge features for each molecule and save
    all the results in H5PY format.
    Args:
    data_dict (dict) contains a hash (key) and 
     the ETDAC matrix (value) of each molecule.
    data_set_name (str, optional) is the name of the H5PY file to be
     created. By default that file is called "data_etdac_matrix.h5".
    """
    
    with h5py.File(data_set_name, 'w') as f:

        # Get node/edge features for the list of molecules
        for hash in data_dict.keys():
#df.to_hdf('data.h5', key='df', mode='w', format='table')
            qm_group = f.create_group(f"sample_{hash}")
            qm_group.create_dataset("ETDAC_matrix", 
                                    data=data_dict[hash].to_records(index=True),  # Preserves index+columns
                                    compression="gzip")
            
            qm_group.attrs["hash"] = hash
            qm_group.attrs["column_name"] = data_dict[hash].columns.name # Save column name
            qm_group.attrs["index_name"] = data_dict[hash].index.name  # Save row name

In [10]:
def load_conditions(conditions_file):
    """
    Load conditions, those are pre-defined by the user
    in an external file and return a dictionary having this
    information.
    """
    parameters= {}
    with open(conditions_file, 'r', encoding="utf-8") as params:
        for param in params:
            param = param.strip()
            parameters[param.split(':')[0].strip()] = eval(param.split(':')[1].strip())
    return parameters

In [11]:
def load_molecules(file_list):
    """
    Return a numpy array having the hash
    and the pdb file name of a pdb list file.
    """
    pdbs_data = []
    with open(file_list, 'r', encoding="utf-8") as listpdb:
        for pdb_file in listpdb:
            pdb_file = pdb_file.strip()
            pdb_file = np.array(pdb_file.split(','))
            pdbs_data.append(pdb_file)
    return np.array(pdbs_data, dtype=h5py.string_dtype(encoding='utf-8'))

In [12]:
#### Functions for node featuring

In [13]:
def pandas_molecule(file_name):
    """
    Load the molecule from pdb format
    and return some of the pdb information
    in pandas format
    """
    # Loading molecule
    atoms = read(file_name, format="proteindatabank")
    # Convert to pandas DataFrame
    atoms_pdb = pd.DataFrame({
        'atom': atoms.get_chemical_symbols(),
        # Optional: add residue info if available in PDB
        'residue': atoms.get_array('residuenames') if 'residuenames' in atoms.arrays else None,
        'x': atoms.positions[:, 0],
        'y': atoms.positions[:, 1],
        'z': atoms.positions[:, 2]
    })
    # Remove blank spaces from 'residue' column
    atoms_pdb['residue'] = atoms_pdb['residue'].str.strip()
    return atoms_pdb

In [14]:
def atom_categorical_feat(pandas_mol):
    """
    Receive a pandas matrix that contains the basic
    information of a pdb file: atom, residue, x, y, z.
    It appends, as new binary columns, the type of atom
    in the following order: 'C', 'N', 'O', 'H' and 'S',
    where 1 if the atom is of that type and 0 otherwise.
    """
    pandas_mol['C'] = (pandas_mol['atom'].eq('C')).astype(int)
    pandas_mol['N'] = (pandas_mol['atom'].eq('N')).astype(int)
    pandas_mol['O'] = (pandas_mol['atom'].eq('O')).astype(int)
    pandas_mol['H'] = (pandas_mol['atom'].eq('H')).astype(int)
    pandas_mol['S'] = (pandas_mol['atom'].eq('S')).astype(int)
    return pandas_mol

In [15]:
def byindex_atom_categorical_feat(pandas_mol, feat_name, index_array):
    """
    Receive a pandas matrix that contains the basic
    information of a pdb file: atom, residue, x, y, z.
    It appends a binary column using feat_name.
    Where values in this new column are 1 if the atom
    (using index of pandas_mol) matches with any value of
    the index_array and 0 otherwise.
    Args:
    pandas_mol (df): pdb file in pandas format
    feat_name (str): name of the new categorical feature
    index_array (np.array): index array to be matched
     with the index values of pandas_mol
    """
    pandas_mol[feat_name] = pandas_mol.index.isin(index_array).astype(int)
    return pandas_mol

In [16]:
def two_aa_categorical_feat(pandas_mol):
    """
    Receive a pandas matrix that contains the basic
    information of a pdb file: atom, residue, x, y, z.
    Differentiate ONLY between two type of residues.
    It appends, two new binary columns, as aa_1 and
    aa_2. This differentiation is only applicable in the
    context of the pair of amino acid model.
    aa_1 and aa_2 corresponds to the two type of amino
    acids.
    """
    pandas_mol['aa_1'] = (
        pandas_mol['residue'] == pandas_mol['residue'].iloc[0]
    ).astype(int)
    pandas_mol['aa_2'] = (
        pandas_mol['residue'] != pandas_mol['residue'].iloc[0]
    ).astype(int)
    return pandas_mol

In [17]:
def stacking_feats(mol_label):
    """
    Stacks all the node features in one single
    pd.Frame recursively.
    Args:
    mol_label (np.array): array having the hash
    and name of the pdb file.
    """
    # load etdac data
    etdac_data = load_etdac_h5py(
        'data_etdac_matrix.h5', 
        mol_label[0]
    )
    return two_aa_categorical_feat(
        byindex_atom_categorical_feat(
            byindex_atom_categorical_feat(
                atom_categorical_feat(
                    pandas_molecule(
                        mol_label[1]
                    )
                ),
                'core',
                etdac_data[1].index.values.astype(int)
            ),
            'virtual',
            etdac_data[1].columns.values.astype(int)
        )
    )

In [18]:
def drop_off_corecolumns(pandas_mol, cols):
    """
    Drop off columns from pandas_mol using cols
    """
    return pandas_mol.drop(columns=cols)

_________________________________

In [19]:
def corexvirt_edge_feature(atoms, params):
    """
    Get the minimal information required for
    GNN context from a molecule regarding 
    connectivity: edges of the adjacency matrix 
    and the pairwise matrix as edge feature.
    From a pdb molecule two np.arrays are created:
    Args:
    atoms from a pdb file (ase.atoms.Atoms)
    Output:
    (edge_list, pairwise_feat) each element in the
     tuple is a np.array.
    edge_list  has shape (2, n_atoms*(n_atoms - 1))
    of a fully connected matrix (i, j) and (j, i)
    excluding diagonal.
    First row is the "Begin" of a connection and
    the second row is the "End" of that connection.
    Since that graph is undirected, half of the 
    information is redundant.
    pairwise_feat is a distance matrix between the 
    non-diagonal elements built with the same order
    architecture of edge_list.
    pairwise_feat has shape (n_atoms*(n_atoms - 1),)
    """

    core = np.where(
        np.array(
            [1 if atom == params
             else 0 for atom in atoms.get_chemical_symbols()]
        )
    )[0]
    
    virtual = np.where(
        np.array(
            [1 for atom in atoms.get_chemical_symbols()]
        )
    )[0]
    
    # All combinatorial pairs
    # core HAS to be a subgroup of virtual 
    rows = np.repeat(core, len(virtual))
    cols = np.tile(virtual, len(core))
    # rows and cols are the positions of core and virtual
    # since virtual are all atoms then the result should 
    #  be the same
    # Shape: (2, core_atoms*virtual_atoms) 
    edge_list = np.vstack([rows, cols])
    # Construct pairwise matrix with the same architecture as edge_list
    pairwise_feat = atoms.get_all_distances()[rows, cols]
    
    return (edge_list, pairwise_feat)

In [20]:
def get_nondiagonals_connectivity(group_1, group_2):
    """
    Create all possible edges between 
     two group of nodes: group_1 and group_2).
    In the options are only consider nondiagonal
    interactions, self-inetractions are deprecated.
    Args:
    group_1 and group 2: np.array
    Output is a np.array e.g.:
    array([[1, 2], [1,3],...])
    """
    return np.column_stack([
        np.repeat(group_1, len(group_2)),
        np.tile(group_2, len(group_1))
    ])

In [21]:
def get_aro_aro_pairlist(mol, fragments):
    """
    Fragments a molecule (RdKit) into the non-covalent
    parts (e.g. two non-covalent interacting amino acids
    are two fragments) and returns a collection of all
    the possible pair atoms between the two aromatic 
    rings of both aromatic amino acids.
    Output:
    a np.array of a list of lists.
    """
    
    # Get only aromatic atoms of each fragment (molecule)
    aa_1 = np.intersect1d(np.array([aa.GetIdx() for aa in mol.GetAromaticAtoms()]), 
                          fragments[0])
    aa_2 = np.intersect1d(np.array([aa.GetIdx() for aa in mol.GetAromaticAtoms()]), 
                          fragments[1])

    # Collecting all the possible edges between aromatic atoms in aa_1 and aa_2
    # Only non-diagonals (a and b can be distinct)
    # Get list of connected (aromatic pi stacking) points (atoms)
    return get_nondiagonals_connectivity(aa_1, aa_2)

In [22]:
def get_sp2_noaromatic_index(mol):
    """
    Returns the ordered np.array with the index
    (positions) of the atoms in mol that have
    hybridation SP2 and are not in a conjugated
    system. However this function is customized
    to include the N from the peptide bond since
    molecules are isolated amino acids and N cannot
    be detected as SP2. Consider this if you are using
    this function in other type of molecules.
    """
    return np.sort(
        np.concatenate((
            np.array([b.GetIdx() for b in mol.GetAtoms() 
                      if (b.GetHybridization() == Chem.HybridizationType().SP3) &
                      (b.GetAtomicNum() == 7)]), # special case for N of peptide bond
            np.array([b.GetIdx() for b in mol.GetAtoms() 
                      if (b.GetHybridization() == Chem.HybridizationType().SP2) &
                      ((b.GetAtomicNum() == 6) | (b.GetAtomicNum() == 8)) 
                      & (~b.GetIsAromatic())]) # SP2 (C or O) atoms
        ),  axis=0)
    )

In [23]:
def get_aro_sp2_pairlist(mol, fragments):
    """
    Fragments a molecule (RdKit) into the non-covalent
    parts (e.g. two non-covalent interacting amino acids
    are two fragments) and returns a collection of all
    the possible pair atoms between the aromatic 
    rings and the peptide bond of both amino acids.
    Output:
    a np.array of a list of lists.
    """

    # Get only aromatic atoms of each fragment (molecule)
    aa_1 = np.intersect1d(np.array([aa.GetIdx() for aa in mol.GetAromaticAtoms()]),
                          fragments[0])
    aa_2 = np.intersect1d(np.array([aa.GetIdx() for aa in mol.GetAromaticAtoms()]), 
                          fragments[1])
    
    # Get only SP2 atoms of each fragment including 
    #  N of peptide bond in isolated amino acids
    sp2_1 = np.intersect1d(get_sp2_noaromatic_index(mol), fragments[0])
    sp2_2 = np.intersect1d(get_sp2_noaromatic_index(mol), fragments[1])

    # Collecting all the possible edges between aromatic atoms in sp2_1 and sp2_2
    # Only non-diagonals (a and b can be distinct)
    # Get list of connected (aromatic-sp2 pi stacking) points (atoms)
    aro_sp2_list1 = get_nondiagonals_connectivity(aa_1, sp2_2)
    aro_sp2_list2 = get_nondiagonals_connectivity(aa_2, sp2_1)
    # Concatenate: full pi-stacking between intermolecular 
    #  sidechain and backbone
    return np.concatenate((aro_sp2_list1, aro_sp2_list2), axis=0)

In [24]:
def get_pairindices_edgelist_asym(edge_list, pairs):
    """
    Return the column positions from edge_list
    where the pair of given values (pairs)
    are connected including redundancies.
    Args:
    edge_list (np.array) is a matrix of connectivity
    (fully connected, including reduncandies and no 
     self-interactions)
    pairs (np.array([0, 1], [0, 2],...)) is a set of
    pair of numbers (nodes) in a np.array that have a 
    connectivity.
    """
    # Check BOTH (a,b) AND (b,a) in one step
    mask = ((edge_list[0][None] == pairs[:, 0][:, None]) & 
            (edge_list[1][None] == pairs[:, 1][:, None])) | \
           ((edge_list[1][None] == pairs[:, 0][:, None]) & 
            (edge_list[0][None] == pairs[:, 1][:, None]))
    # Get indices where either condition is True
    return np.where(mask.any(axis=0))[0]

In [25]:
def get_pairindices_edgelist(edge_list, pairs):
    """
    Return the column positions from edge_list
    where the pair of given values (pairs)
    are connected including redundancies.
    Args:
    edge_list (np.array) is a matrix of connectivity
    (fully connected, including reduncandies and no 
     self-interactions)
    pairs (np.array([0, 1], [0, 2],...)) is a set of
    pair of numbers (nodes) in a np.array that have a 
    connectivity.
    """
    return np.sort(
        np.column_stack(
            [
                np.where((edge_list[0] == pairs[:, 0][:, None]) & 
                         (edge_list[1] == pairs[:, 1][:, None]))[1],
                np.where((edge_list[1] == pairs[:, 0][:, None]) & 
                         (edge_list[0] == pairs[:, 1][:, None]))[1]
            ]
        ), axis=1 # sort each row
    )

In [26]:
def customint_edge_categorical_feat(e_feat_shape, edge_list, if_pair_list):
    """
    Creates a edge feature array following a categorical
     condition using an already "conditioned" pair list.
    This function follows a similar structure as in the 
      typeint_edge_categorical_feat() function.
    In this case customization comes from the fact that
    if_pair_list is already conditioned by the user needs.
    Args:
    e_feat_shape (tuple) is the shape of the edge
     feature array.
    edge_list (np.array) is the array of connectivity
     for a undirected graph (all non-diagonals).
    Read more about it in the default_edge_feature() 
     function.
    if_pair_list (np.array) is the array of connectivity 
    having shape (numer_of_acceptable_conditions, 2).
    int_type (float) is the type of "covalent" bond
     interaction defined in the rdkit.Chem library.
    Read more about it in the type_int_categorical_feat()
     function.
    """

    # Initializing the edge feature array
    edge_feature = np.zeros(e_feat_shape, dtype=int)
       
    # Get the column positions of edge_list where each if_pair_list
    #  matches including redundancies (i, j) and (j, i).
    #column_pos = get_pairindices_edgelist(edge_list, if_pair_list)
    column_pos = get_pairindices_edgelist_asym(edge_list, if_pair_list)
    
    # Assign the value 1 for the indices in edge_feature that are 
    #  the elements in column_pos
    edge_feature[np.sort(column_pos.flatten())] = 1

    return edge_feature

In [27]:
def cross_edge_feat_pi_stack(weight_efeat, cat_edge_feat, dist_condition):
    """
    Create a categorical cross-feature flag using the information of 
    a weight edge feature array and a categorical edge feature both
    with the same dimensions and information order.
    The crossing of these two edge feature arrays is done by a
    third condition (nummerical one).
    Args:
    weight_efeat (np.array) is a nummerical array,
    cat_edge_feat (np.array) is a categorical array and
    dist_condition (float) is a number used as threshold:
    when values in weight_efeat are lower than dist_condition
    and fullfil the condition of cat_edge_feat then assing
    value of 1 in the cr_edge_ft, else 0.
    Output (np.array) is the categorical array cr_edge_ft
    """
    # Initializing the cross-feature flag array
    cr_edge_ft = np.zeros(weight_efeat.shape, dtype=int)

    # 
    cr_edge_ft[np.where(
        weight_efeat[np.where(
            cat_edge_feat)[0]] < dist_condition)[0]
    ] = 1
    return cr_edge_ft

In [28]:
def clip_array(array, min_bound, max_bound):
    """
    Uses min_bound and max_bound as limit, outliers
    are forced to take any of the bound values.
    """
    return np.clip(array, min_bound, max_bound)

In [29]:
def clip_norm(array, thr_low, thr_up):
    """
    Uses a lower and upper thresholds to force outliers
    to set in these limits and normalize values.
    Another normalization is done only to reinforce bounds.
    """
    # Increasing the bounds to do a soft clip
    # minimum covalent bond distance ~0.98 
    # 0.039 is an aribitrary decision and requires
    # a reinforce in the normalization applied later
    thr_low = thr_low - 0.039 # for covalent bonds
    # thr_up*1.0416 equivalent to thr_up/0.96
    thr_up = np.round(thr_up*1.0416, 2) # for noncovalent bonds
    # Normalization [0, 1] after clipping: 
    #  outliers below 0.003 and above 0.96
    return minmax_norm( #second normalization
        clip_array( #clipping [0,1] to reinforce bounds
            minmax_norm( # first normalization
                clip_array( #clipping [thr_low, thr_up]
                    array, thr_low, thr_up
                ),
                thr_low, thr_up
            ),
            0, 1
        ),
        0, 1
    )

In [30]:
def minmax_norm(array_values, min, max):
    """
    Returns a normalized matrix using the minimum and the 
    maximium values set by user.
    """
    return (array_values - min) / (max - min + 1e-6)

In [31]:
def stack_categorical_edge_feats(mol, pairwise_feat, edge_list):
    """
    Stack all the categorical edge feature arrays
    including the cross-feature flags using a 
    pairwise matrix (weight feature).
    Output (np.array),e.g.: stacked_edge_features
    where eight arrays are stacked.
    stacked_edge_features[:, 0] is pairwise_feat
    stacked_edge_features[:, 1] is single bond
    stacked_edge_features[:, 2] is double bond
    stacked_edge_features[:, 3] is aromatic bond
    stacked_edge_features[:, 4] is aromatic-aromatic
      pi-stacking 
    stacked_edge_features[:, 5] is aromatic-sp2
      pi-stacking
    stacked_edge_features[:, 6] is aromatic-aromatic
      pi-stacking lower than 6.5 Angstroms
    stacked_edge_features[:, 7] is aromatic-sp2
      pi-stacking lower than 5.5 Angstroms
    """
    # Get list of connected (covalent bonds) points (atoms)
    #pair_list = np.array([[b.GetBeginAtomIdx(), b.GetEndAtomIdx()] for b in mol.GetBonds()])

    # Normalize the pairwise matrix using minmax method and 
    #  forcing outliers to set into the bounds decided by user
    # Minimum distance 0.98 and maximum 12 Angstroms
    # clip_norm() does a double minmax normalization forcing outliers
    pairwise_feat_norm = clip_norm(pairwise_feat, 0.98, 12)
    # creating edge categorical features (covalent bond type)
    #single_efeat = typeint_edge_categorical_feat(mol, pairwise_feat.shape, edge_list, pair_list, 1) # single
    #double_efeat = typeint_edge_categorical_feat(mol, pairwise_feat.shape, edge_list, pair_list, 2) # double
    #aromatic_efeat = typeint_edge_categorical_feat(mol, pairwise_feat.shape, edge_list, pair_list, 1.5) # aromatic

    # Idea for pi_stacking (conditioned only for two molecules)
    fragments = [np.array(frag) for frag in Chem.GetMolFrags(mol, asMols=False)]
    
    # Get list of connected (aromatic pi stacking) points (atoms)
    aro_aro_pair_list = get_aro_aro_pairlist(mol, fragments)

    # creating edge categorical features (non covalre's the correctedent bond type)
    aroaro_pi_efeat = customint_edge_categorical_feat(
    pairwise_feat.shape, edge_list, aro_aro_pair_list) # aromatic-aromatic pi stacking

    # Get list of connected (aromatic side chain/backbone pi stacking) points (atoms)
    aro_sp2_pair_list = get_aro_sp2_pairlist(mol, fragments)

    # creating edge categorical features (non covalent bond type)
    arosp2_pi_efeat = customint_edge_categorical_feat(
    pairwise_feat.shape, edge_list, aro_sp2_pair_list) # aromatic-sp2 pi stacking

    # Create cross-feature flag between pi-stacking interaction mediated by some distance threshold
    cross_feat_flag_1 = cross_edge_feat_pi_stack(pairwise_feat, aroaro_pi_efeat, 6.5) # aromatic-aromatic < 6.5
    cross_feat_flag_2 = cross_edge_feat_pi_stack(pairwise_feat, arosp2_pi_efeat, 5.5) # aromatic-sp2 < 5.5

    return np.column_stack([
        pairwise_feat_norm,
        #single_efeat,
        #double_efeat,
        #aromatic_efeat,
        aroaro_pi_efeat,
        arosp2_pi_efeat,
        cross_feat_flag_1,
        cross_feat_flag_2
    ])

In [32]:
def save_nodesnedges_h5py(molecules_info, mol_params, etdac_data, data_set_name="data_metadata.h5"):
    """
    Get the node/edge features for each molecule and save
    all the results in H5PY format.
    Args:
    molecules_info (np.array) contains a hash and 
    the name of each molecule.
    mol_params (dictionary) are the global parameters
    for that set of molecules in molecules_info.
    data_set_name (str, optional) is the name of the H5PY file to be
    created. By default that file is called "data_metadata.h5".
    """
    
    with h5py.File(data_set_name, 'w') as f:
        # Save the global parameters as attributes once for thw file
        for key, value in mol_params.items():
            f.attrs[key] = value

        # Get node/edge features for the list of molecules
        for mol_label in molecules_info:
            # molecule is mol_label[1]

            # Get array 1: node features
            node_feats = np.array(
                drop_off_corecolumns(
                    stacking_feats(mol_label),
                    mol_params['cols']
                )
            )
    
            # Load structural information
            atoms = read(mol_label[1], format="proteindatabank")
            # Get array 2: edge list
            edge_list, pairwise_feat = corexvirt_edge_feature(
                atoms, mol_params['core_atm']
            )

            # Load molecule using RdKit
            mol = Chem.MolFromPDBFile(mol_label[1], removeHs=False)
            # Get array 3: edge features
            print(mol_label[0])
            edge_features = np.column_stack([
                etdac_data['sample_'+mol_label[0]].to_numpy().ravel(),
                stack_categorical_edge_feats(mol, pairwise_feat, edge_list)
            ])
            
            # Create a group for the sample molecule
            mol_group = f.create_group(f"sample_{mol_label[0]}")
            
            # Save arrays (with compression to reduce file size)
            mol_group.create_dataset("Node_features", data=node_feats, compression="gzip")
            mol_group.create_dataset("Edge_list", data=edge_list, compression="gzip")
            mol_group.create_dataset("Edge_features", data=edge_features, compression="gzip")
            
            # Save individual attributes
            mol_group.attrs["name"] = mol_label[1]
            mol_group.attrs["hash"] = mol_label[0]
        

In [33]:
# Load the parameters that will be used in the node feature extraction
parameters = load_conditions('conditions_etdac.txt')

In [34]:
# Load the hash and molecule name that will be used in the node/edge feature extraction
file_list = "list_100pdb_files.txt"
molecules_labels = load_molecules(file_list)

In [35]:
etdac_data = load_etdac_h5py('data_etdac_matrix.h5', molecules_labels[10][0])

In [36]:
all_etdac_data = loadseveral_nodesnedges_h5py('data_etdac_matrix.h5')

In [ ]:
all_etdac_data['sample_'+molecules_labels[43][0]].to_numpy().ravel()

In [37]:
save_nodesnedges_h5py(molecules_labels, parameters, all_etdac_data, data_set_name="data_etdac_metadata.h5")

f00
f01
f02
f03
f04
f05
f06
f07
f08
f09
f10
f11
f12
f13
f14
f15
f16
f17
f18
f19
f20
f21
f22
f23
f24
f25
f26
f27
f28
f29
f30
f31
f32
f33
f34
f35
f36
f37
f38
f39
f40
f41
f42
f43


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 448 and the array at index 1 has size 810

In [ ]:
molecules_labels, parameters, data_set_name=h5file

In [ ]:


import csv as csv
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Save all the molecule list in H5PY format
h5file = "data_100mol_test.h5"
save_nodesnedges_h5py(molecules_labels, parameters, data_set_name=h5file)

In [ ]:
## test purposes
rel_list = []
tmp_keys = [key for key in heatmap.keys()][10:58] #from 3.5 to 9.0 A
for ii in tmp_keys: 
    rel_list.append(heatmap[ii].max().max())
relmax = max(rel_list)
relmin = min(rel_list)
relmin, relmax

##### Plot electron transition density atomic contribution (ETDAC) matrices

In [ ]:
fig, ax = plt.subplots(figsize=(17,10)) 
test = (heatmap['f13'] - heatmap['f13'].min().min())/(heatmap['f13'].max().max() - heatmap['f13'].min().min())
sns.heatmap(test, annot=False, cmap='Oranges', vmin=0, vmax=1, ax=ax)

#### Definition of amino acids using range of atoms
##### Example is the same Phe --- Tyr

##### Set atoms

In [ ]:
atomAi = 0
atomAf = 22
atomBi = 23
atomBf = 46

In [ ]:
atomAi, atomAf, atomBi, atomBf

#### Delitimation of atoms of the aromatic rings

In [ ]:
atomFi = 6
atomFf = 11
atomYi = 29
atomYf = 33
atomYf2 = 35

#### Calculating the 4 (more) terms of the transition intensities

In [ ]:
inter_fosce = []
all_fosce = []
FYpi_inter_fosce = []
YFpi_inter_fosce = []
aropi_inter_fosce = []
keys = []

for key in heatmap.keys():
    keys.append(key)
    row_cond_1, col_cond_1 = lambda i: i > atomAf, lambda i: i < atomBi
    row_cond_2, col_cond_2 = lambda i: i <= atomAf, lambda i: i >= atomBi
    inter_fosce.append(
        crop_heatmap_byatm(heatmap[key], row_cond_1, col_cond_1).sum().sum() +
        crop_heatmap_byatm(heatmap[key], row_cond_2, col_cond_2).sum().sum()
    )
    row_cond_1 = lambda i: i >= atomFi and i <= atomFf
    col_cond_1 = lambda i: (i >= atomYi and i <= atomYf) or i == atomYf2
    FYpi_inter_fosce.append(
        crop_heatmap_byatm(heatmap[key], row_cond_1, col_cond_1).sum().sum()
        )
    row_cond_1 = lambda i: (i >= atomYi and i <= atomYf) or i == atomYf2
    col_cond_1 = lambda i: i >= atomFi and i <= atomFf
    YFpi_inter_fosce.append(
        crop_heatmap_byatm(heatmap[key], row_cond_1, col_cond_1).sum().sum()
        )
    
    aropi_inter_fosce = [FYpi_inter_fosce[i] + YFpi_inter_fosce[i] for i in range(len(inter_fosce))]
    
    all_fosce.append(heatmap[key].sum().sum())
    
intra_fosce = [all_fosce[i] - inter_fosce[i] for i in range(len(inter_fosce))]

In [ ]:
max(inter_fosce), max(intra_fosce), max(all_fosce)

In [ ]:
dfftotal_fosce = pd.DataFrame({'hash': [i for i in keys],\
                               'inter_fosce': [i for i in inter_fosce],\
                               'intra_fosce': [i for i in intra_fosce],\
                               'all_fosce': [i for i in all_fosce],\
                               'FY_pi':[i/max(inter_fosce) for i in FYpi_inter_fosce],\
                               'YF_pi':[i/max(inter_fosce) for i in YFpi_inter_fosce],\
                               'pi_pi':[i/max(inter_fosce) for i in aropi_inter_fosce],\
                               'abs_pi_pi':[i for i in aropi_inter_fosce]
                              })

In [ ]:
dfftotal_fosce

#### Data to be saved

In [ ]:
plt.rc('font', size=26)
ax = dfftotal_fosce.loc[10:55,:].plot(
    x="hash",
    y=["inter_fosce","intra_fosce","abs_pi_pi", "all_fosce"],
    kind="line",
    figsize=(16, 12))
ax.set_xlabel('Sample unique identificator')
ax.set_ylabel('Transition intensity')

plt.show()

#### That's it :)